In [1]:
%load_ext lab_black
%cd ..

/home/shim/cev/dl/log-analytics


In [16]:
import pickle
import pandas as pd
from datasets.dataset_ver5 import MyDatasetVer5
from tqdm import tqdm
import sys
from collections import defaultdict

In [3]:
df = pd.read_csv("data/ori/train.csv")

In [4]:
# {텍스트: 레벨: 개수}
db = {}

In [6]:
with tqdm(total=len(df), ncols=100, file=sys.stdout) as t:
    for i, (level, full_log) in enumerate(zip(df.level, df.full_log)):
        # text = MyDatasetVer5.refine_data(full_log)
        text = full_log
        if text not in db:
            db[text] = {}
        if level not in db[text]:
            db[text][level] = {"cnt": 0, "list": []}
        db[text][level]["cnt"] += 1
        db[text][level]["list"].append(i)

        t.update()

100%|███████████████████████████████████████████████████| 472972/472972 [00:01<00:00, 373439.30it/s]


In [7]:
keys = list(db.keys())

In [8]:
len(keys), len(df)

(421079, 472972)

In [9]:
duples = {}
for i, key in enumerate(keys):
    if len(db[key]) != 1:
        K = tuple(sorted(list(db[key].keys())))
        if K not in duples:
            duples[K] = []
        duples[K].append(i)

In [10]:
duples.keys()

dict_keys([(0, 1), (0, 5), (3, 5), (0, 1, 5), (1, 5), (0, 3)])

In [11]:
duples[(0, 1)][:5]

[392, 903, 1002, 1158, 1216]

In [12]:
db[keys[392]]

{0: {'cnt': 2, 'list': [562, 562]}, 1: {'cnt': 2, 'list': [383177, 389360]}}

지금 상황에서는 중복되는 level을 갖는 데이터가 정확하게 어떤 level인지 알 방법이 없음.

그러므로 수가 많은 쪽으로 가도록 하자

In [13]:
outdb = {"level": [], "text": []}
with tqdm(total=len(keys), ncols=100, file=sys.stdout) as t:
    for key in keys:
        d = db[key]
        if len(list(d.keys())) > 1:
            # 가장 개수가 많은 level을 선택
            cnts = sorted([(d[k]["cnt"], k) for k in d.keys()], reverse=True)

            # 만약 개수가 같은게 있다면 가장 level이 작은 것을 선택
            max_cnt = cnts[0][0]
            same_ks = list(filter(lambda x: x[0] == max_cnt, cnts))
            level = same_ks[-1][1]
        else:
            level = list(d.keys())[0]

        outdb["level"].append(level)
        outdb["text"].append(key)
        t.update()

100%|███████████████████████████████████████████████████| 421079/421079 [00:00<00:00, 954321.56it/s]


In [14]:
temp = defaultdict(int)
for level in outdb["level"]:
    temp[level] += 1

In [15]:
temp  # 각 레벨별 데이터의 개수

defaultdict(int, {0: 280793, 1: 134195, 3: 4219, 5: 1842, 2: 12, 4: 10, 6: 8})

생각외로 IP주소(내부망인지 외부망인지 등에 따라서 차이가 있을 수 있기 때문), 시간(어느 시간대에 공격이 발생할 활률이 높은지 등의 의미가 있을 수 있음) 등도 중요한 정보 중 하나가 될 수 있겠다고 생각이 된다.

그러니까 원본 데이터를 그대로 입력해보자(그래도 중복은 제거해서)

In [17]:
!mkdir data/ver6

In [19]:
with open("data/ver6/train.pkl", "wb") as f:
    pickle.dump(outdb, f)

---

# 테스트 데이터

In [20]:
df = pd.read_csv("data/ori/test.csv")

In [25]:
full_logs = [full_log for full_log in df.full_log]

In [28]:
db = {}
with tqdm(total=len(df), ncols=100, file=sys.stdout) as t:
    for id, full_log in zip(df.id, df.full_log):
        # text = MyDatasetVer5.refine_data(full_log)
        text = full_log
        if text not in db:
            db[text] = []
        db[text].append(id)

        t.update()

100%|█████████████████████████████████████████████████| 1418916/1418916 [00:02<00:00, 609184.17it/s]


In [31]:
len(db.keys()), len(df)  # db: {full_log: "list of ids"}

(1095951, 1418916)

In [32]:
with open("data/ver6/test.pkl", "wb") as f:
    pickle.dump(db, f)

---

# Level7 validation 데이터

In [33]:
df = pd.read_csv("data/ori/validation_sample.csv")

In [34]:
full_logs = [full_log for full_log in df.full_log]

In [35]:
full_logs

['type=ANOM_PROMISCUOUS msg=audit(1600402733.466:4503): dev=enp2s0 prom=256 old_prom=0 auid=4294967295 uid=0 gid=0 ses=4294967295 type=SYSCALL msg=audit(1600402733.466:4503): arch=c000003e syscall=54 success=yes exit=0 a0=c a1=107 a2=1 a3=7f856aed1140 items=0 ppid=1 pid=12152 auid=4294967295 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=4294967295 comm="W#01-enp2s0" exe="/usr/sbin/suricata" subj=system_u:system_r:unconfined_service_t:s0 key=(null) type=PROCTITLE msg=audit(1600402733.466:4503): proctitle=2F7362696E2F7375726963617461002D63002F6574632F73757269636174612F73757269636174612E79616D6C002D2D70696466696C65002F7661722F72756E2F73757269636174612E706964002D6900656E70327330',
 'oscap: msg: "xccdf-result", scan-id: "0001600739632", content: "ssg-centos-7-ds.xml", title: "Prevent Log In to Accounts With Empty Password", id: "xccdf_org.ssgproject.content_rule_no_empty_passwords", result: "fail", severity: "high", description: "If an account is configured for pass

In [36]:
with open("data/ver6/valid-level7.pkl", "wb") as f:
    pickle.dump(full_logs, f)

In [37]:
A = {1: 2, 2: 3, 3: 4}

In [40]:
A.keys()

dict_keys([1, 2, 3])